## networkz

Nettverksanalyse gjort klar for bruk med geopandas.

Pakken inneholder tre former for nettverksanalyse:
- od_cost_matrix: rask beregning av reisetid/distanse
- shortest_path: treigere variant som returnerer rutene som linjer
- service_area: beregner området som kan nås innen en eller flere reisetider/distanser

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

import os
while "networkz" not in os.listdir():
    os.chdir("../")

pd.options.mode.chained_assignment = None # ignorerer midlertidig SettingWithCopyWarning

import networkz as nz
nz.__version__

In [ ]:
# statisk eller interaktiv kartlegging. Interaktivt tar mer tid/plass, så går for statisk her.
statisk = True
def kartlegg(gdf, kolonne=None, statisk=statisk, legend=True, **qwargs):
    if statisk:
        fig, ax = plt.subplots(1, figsize=(9, 9))
        ax.set_axis_off()
        gdf.plot(column=kolonne, ax=ax, **qwargs)
    else:
        display(gdf.explore(column=kolonne, **qwargs))

Aller først må man ha en GeoDataFrame med punktdata.

Her er 1000 tilfeldige adresser i Oslo:

In [ ]:
punkter = nz.les_geopandas("ssb-prod-dapla-felles-data-delt/GIS/Vegnett_benchmark/tilfeldige_adresser_100/tilfeldige_adresser_100.parquet")
punkter

Man starter med Graf:

In [ ]:
G = nz.Graf()
G

Nå er det nyeste klargjorte vegnettverket lest inn fra fellesbøtta i Dapla.

Parametrene som er printet over kan endres. Mer om det under.

Først demo. Velger ut Oslo for å få det mye raskere:

In [ ]:
G = nz.Graf(kommuner="0301")

In [ ]:
# lager først to små utvalg
n = 10
fra = punkter.sample(n).reset_index(drop=True)
fra["ny_idx"] =  ["fra"+str(x) for x in range(n)] # string-index for illustrasjons skyld
til = punkter.sample(n).reset_index(drop=True)
til["ny_idx"] =  ["til"+str(x) for x in range(n)]

### shortest_path
Korteste rute mellom ett/flere startpunkter til ett/flere sluttpunkter.

In [ ]:
korteste_ruter = G.shortest_path(startpunkter = fra,
                                 sluttpunkter = til,
                                 id_kolonne = "ny_idx")

korteste_ruter

In [ ]:
kartlegg(korteste_ruter[~korteste_ruter.minutter.isna()])

tell_opp=True for å telle antall ganger hver veglenke blir brukt:

In [ ]:
relevante_veglenker = G.shortest_path(startpunkter = punkter.sample(150), 
                                      sluttpunkter = punkter.sample(150),
                                      tell_opp=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 15))
ax.set_axis_off()
ax.set_title("Antall ganger hver veglenke ble brukt", fontsize = 18)
relevante_veglenker["geometry"] = relevante_veglenker.buffer(20)
relevante_veglenker.plot("antall", scheme="NaturalBreaks", cmap="RdPu", k=7, legend=True, alpha=0.8, ax=ax)

radvis=True for å beregne fra startpunkt 1 til sluttpunkt 1, fra startpunkt 2 til sluttpunkt 2 osv:

In [ ]:
korteste_ruter = G.shortest_path(startpunkter = fra, sluttpunkter = til, id_kolonne = "ny_idx",
                                 radvis=True)

korteste_ruter

### od_cost_matrix: 

Rask beregning av reisetid/distanse.

In [ ]:
od = G.od_cost_matrix(startpunkter = punkter, 
                      sluttpunkter = punkter, 
                      id_kolonne = "idx",
                      )
od

dist_node_start og dist_node_slutt er avstanden til nærmeste node fra start- og sluttpunktene. Kjekt for feilsøking.

Man kan også få returnert rette linjer:

In [ ]:
od = G.od_cost_matrix(startpunkter = punkter.sample(1), 
                      sluttpunkter = punkter, 
                      id_kolonne = "idx",
                      linjer=True)
kartlegg(od, "minutter", scheme="Quantiles", legend=True)

radvis=True for å beregne fra ett til ett punkt av gangen.

In [ ]:
od = G.od_cost_matrix(startpunkter = fra, sluttpunkter = til, id_kolonne = "ny_idx",
                      radvis = True)
od

Med destination_count=1 får man bare raskeste/korteste reise for hvert startpunkt (beklager forresten blanding av norske og engelske parametre. Mange av parametrene er bare kopiert fra ArcGIS Pro).

In [ ]:
od = G.od_cost_matrix(startpunkter = fra, sluttpunkter = til, id_kolonne = "ny_idx",
                      destination_count = 1,
                      )
od

Bruk cutoff for å kun få reisene under en viss kostnad: 

In [ ]:
od = G.od_cost_matrix(startpunkter = punkter, sluttpunkter = punkter, id_kolonne = "idx",
                      cutoff = 5
                      )
od

Man kan få både meter og minutter (OBS: tar dobbelt så lang tid). 

In [ ]:
G.kostnad = ["minutter", "meter"]

od = G.od_cost_matrix(startpunkter = punkter,
                      sluttpunkter = punkter,
                      id_kolonne = "idx")
od

In [ ]:
G.kostnad = "minutter"

### service_area
Finn området som kan nås innen en viss tid/distanse/annet (impedance, som det heter i ArcGIS Pro).

Her får man området som kan nås innen fem minutter for fem tilfeldige punkter:

In [ ]:
service_areas = G.service_area(startpunkter = punkter.sample(5),
                               impedance = 5, # antall minutter/meter
                               id_kolonne = "idx")
service_areas

Sånn er ett av områdene ut:

In [ ]:
service_areas.sample(1).plot()

Man kan også ha flere impedances:

In [ ]:
service_areas = G.service_area(startpunkter = punkter.sample(1),
                               impedance = [10,9,8,7,6,5,4,3,2,1], # antall minutter/meter/annet
                               id_kolonne="idx")
kartlegg(service_areas, G.kostnad)

### Sykkel og fot

Reisetider for sykkel og til fots er basert på et eget vegnett som inkluderer fortau, stier og lignende. 

Farten er satt til 4 km/t til fots og 20 km/t for sykkel. I tillegg gis det tidsstraff for oppoverbakker og tidsbonus for nedoverbakker.

La oss sammenligne reisetidene:

In [ ]:
import pandas as pd
resultater = pd.DataFrame()
for kjoretoy in ["fot", "sykkel", "bil"]:
    
    G = nz.Graf(kjoretoy=kjoretoy, kostnad=["minutter", "meter"], kommuner="0301")

    od = G.od_cost_matrix(startpunkter = fra, sluttpunkter = til, id_kolonne = "idx")
    
    od["kjoretoy"] = kjoretoy
    
    resultater = pd.concat([resultater, od], ignore_index=True)

resultater["km"] = resultater.meter/1000
resultater["km_t"] = (resultater.meter/1000) / (resultater.minutter/60)

gruppert = resultater.groupby("kjoretoy").agg(minutter_mean = ("minutter", "mean"),
                                              km_mean = ("km", "mean"),
                                              km_t_mean = ("km_t", "mean"))
gruppert

Med sykkel og til fots kan man ferdes en del steder det ikke er lov å kjøre. Det er også noen steder det kun regnes som lov/mulig å gå (sti, gangveg, fortau) eller sykle (sykkelfelt, sykkelveg).

Når sykkel/fot, blir raskeste rute gjerne nærmere korteste rute. Fordi farten er jevnere, siden sykler og mennesker, heldigvis, som regel ikke kommer opp i livsfarlige hastigheter som 60 og 80 km/t. 

Rutene for sykkel og fotgjengere følger derfor oftere mindre gater/veger enn større motorveier o.l.

Sykkelrutene følger også oftere de større gatene enn fotrutene. Det er fordi stigning påvirker reisetiden mer for syklister enn forgjengere.

In [ ]:
import pandas as pd
utvalg = punkter.sample(150)
resultater = pd.DataFrame()
for kjoretoy in ["bil", "sykkel", "fot"]:
    
    G = nz.Graf(kjoretoy=kjoretoy, kostnad="meter", kommuner="0301")
    
    mest_brukte_gater = G.shortest_path(startpunkter = utvalg, sluttpunkter = utvalg, 
                                     tell_opp=True)
    
    mest_brukte_gater["geometry"] = mest_brukte_gater.buffer(7)
    
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.set_axis_off()
    ax.set_title(f"Antall ganger brukt. Kjøretøy: {kjoretoy}", fontsize = 16)
    mest_brukte_gater.plot("antall", scheme="NaturalBreaks", cmap="RdPu", k=7, legend=True, alpha=1, ax=ax)

Dataene for sykkel/fot er ikke perfekte. La oss ta en nærmere titt på området rundt Akersveien:

In [ ]:
from shapely.wkt import loads
akersveien = gpd.GeoDataFrame({"geometry": gpd.GeoSeries(loads("POINT (10.7476913 59.9222196)"))}, crs=4326).to_crs(25833)
akersveien["geometry"] = akersveien.buffer(500)
punkter_rundt_akersveien = punkter.sjoin(akersveien)

resultater = pd.DataFrame()
for kjoretoy in ["fot", "sykkel"]:

    G = nz.Graf(kjoretoy=kjoretoy, kostnad="minutter", kommuner="0301")
    
    korteste_ruter = G.shortest_path(startpunkter = punkter_rundt_akersveien, sluttpunkter = punkter_rundt_akersveien, id_kolonne = "idx")
    
    korteste_ruter["kjoretoy"] = kjoretoy
    
    resultater = gpd.GeoDataFrame(pd.concat([resultater, korteste_ruter], axis=0, ignore_index=True), geometry="geometry", crs=25833)
    
kartlegg(resultater, "kjoretoy", cmap="bwr")

Vår frelsers gravlund skulle vært mulig å gå gjennom. Statens vegvesen har kombinert sine vegdata med OpenStreetMap for å lage nettverket for sykkel/fot, men filtreringen av OpenStreetMap-dataene er for streng. Det skal muligens fikses i løpet av 2023.

Legg også merke til at et par av punktene starter inni Gamle Aker kirkegård. Det er fordi den ene adressen (i Telthusbakken) er nærmest vegen inni kirkegården, selv om det er et gjerde og høydeforskjell mellom. Reisetiden til fots blir da en god del mer hvis man skal østover mot Grünerløkka.

Det er lagt til en tidsstraff (hvis kostnaden er minutter) for oppoverbakker, og tidsbonus for nedoverbakker.

Har prøvd å matche tidene oppmot google maps (kunne trengt en kvalitetssjekk). For sykkel legges det som default til 23 prosent på tiden per prosent stigning. For fotreiser er default 5 prosent. 

Hvis negativ stigningsprosent, logtransformeres stigningsprosenten (uten minusfortegn). Det fordi luftmotstanden gjær at farten ikke øker lineært når det blir brattere, og fordi man man gjerne må bremse når bratt.

In [ ]:
storo = gpd.GeoDataFrame({"geometry": gpd.GeoSeries(loads("POINT (10.7777979 59.9451632)"))}, crs=4326).to_crs(25833)
storo["idx"] = "storo"
grefsenkollen = gpd.GeoDataFrame({"geometry": gpd.GeoSeries(loads("POINT (10.8038165 59.9590036)"))}, crs=4326).to_crs(25833)
grefsenkollen["idx"] = "grefsenkollen"

G = nz.Graf(kjoretoy="sykkel")

oppover = G.shortest_path(startpunkter = storo, sluttpunkter = grefsenkollen, id_kolonne = "idx")
nedover = G.shortest_path(startpunkter = grefsenkollen, sluttpunkter = storo, id_kolonne = "idx")
nz.gdf_concat([oppover, nedover])

In [ ]:
G = nz.Graf(kjoretoy="sykkel", kostnad="minutter")
med_sykkel = G.shortest_path(startpunkter = storo, sluttpunkter = grefsenkollen, id_kolonne = "idx")
med_sykkel["hva"] = "sykkel"

G = nz.Graf(kjoretoy="fot", kostnad="minutter")
til_fots = G.shortest_path(startpunkter = storo, sluttpunkter = grefsenkollen, id_kolonne = "idx")
til_fots["hva"] = "fot"

kartlegg(nz.gdf_concat([med_sykkel, til_fots]), "hva", cmap="bwr")

### Mer om Graf-classen

De som kjenner til koseptet graf i nettverkssammenheng, stusser nok over at Graf-classen som brukes her ikke egentlig er en graf. Navnet Graf er brukt fordi det er en konvensjon i nettverksanalyse i Python, R (og sikkert flere språk) å starte nettverksanalysen med å initiere en class med navn Graph, som er en faktisk graf. Her lages grafen først inni nettverksbereningene, for å få med lenker fra start- og sluttpunktene til nodene i nettverket. For å unngå forvirring, kunne man byttet navn på classen Graf. Tips tas imot med takk. 

Når man kjører Graf(), lastes det nyeste klargjorte vegnettet fra fellesbøtta inn som en GeoDataFrame og lagres i attributten 'nettverk':

In [ ]:
G = nz.Graf()
G.nettverk.head(3)

Man kan velge eldre vegnett (tilbake til 2019):

In [ ]:
G = nz.Graf(2021)
G.aar

Man kan også bruke egne vegnett. Det bør kjøres gjennom funksjonen lag_nettverk for å få riktige kolonnenavn osv.

Man kan enten spesifise kolonnenavnene:

In [ ]:
veger = gpd.read_parquet(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\data\vegdata\veger_oslo_og_naboer_2021.parquet")

nettverk = nz.lag_nettverk(veger,
                            source = "fromnodeid",
                            target = "tonodeid",
                            minutter = ("drivetime_fw", "drivetime_bw")
                            )
                     
G = nz.Graf(nettverk=nettverk)
G.nettverk.head()

Eller lage et retningsløst nettverk bare basert på linjegeometrien. 

Da bør/må man også endre et par parametre i Graf:
- directed bør være False, hvis ikke man vet at linjegeometrien er i riktig rekkefølge. Med directed=False kan man bevege seg i begge retninger langs nettverket. 
- hvis kjoretoy='bil', må kostnad være "meter" eller en kolonne som finnes i dataene. Hvis sykkel eller fot, beregnes minuttene fra meter-kolonnen.

In [ ]:
veger_kun_geom = veger[["geometry"]]

nettverk = nz.lag_nettverk(veger_kun_geom)

G = nz.Graf(nettverk=nettverk, 
            directed=False,
            kostnad="meter")

G.nettverk.head()

Det er altså mulig å velge minutter som kostnad hvis kjøretøyet er sykkel eller fot: 

In [ ]:
G = nz.Graf(nettverk=nettverk, 
            directed=False,
            kjoretoy="sykkel",
            kostnad="minutter")
G.nettverk.head()

Man kan velge ut bare relevante kommuner for å få det mye raskere.

For eksempel bare Oslo og nabokommunene:

In [ ]:
import kommfylk
oslos_naboer = kommfylk.nabokommuner("0301")
oslos_naboer

In [ ]:
oslo_og_naboer = ["0301"] + oslos_naboer

G = nz.Graf(kommuner = oslo_og_naboer)

G.nettverk.KOMMUNENR.value_counts()

Eller hvis man vil loope dette for hver kommune, evt med nabokommuner:

In [ ]:
for kommnr in kommfylk.kommuner_fra_api(2022):
    G = nz.Graf(kommuner = kommnr)
    # og så velge ut start- og sluttpunkter fra relevant kommune, så kjøre nettverksanalyse
    
for kommnr, naboer in kommfylk.nabokommuner(aar=2022).items():
    G = nz.Graf(kommuner = [kommnr] + naboer)

## Regler for nettverksanalysen

Graf-classen inneholder regler for hvordan nettverksanalysen skal gjøres:

In [ ]:
G = nz.Graf()
G

Mer info om dem her:

In [ ]:
G.info()

Man kan tilpasse grafen sin når man initierer Graf().

In [ ]:
G = nz.Graf(aar = 2022, 
            kostnad = "meter",
            directed=False,
            kjoretoy = "sykkel",
            search_tolerance = 500,
            dist_faktor = 50
            )
G

De fleste attributtene kan også endres etterpå:

In [ ]:
G.nettverk = G.nettverk[G.nettverk.KOMMUNENR=="0301"]
G.kostnad = ["minutter", "meter"]
G.directed = False
G.turn_restrictions = True
G.search_tolerance = 200
G.dist_faktor = 10
G.kost_til_nodene = False
G.fart = 5
G

Det er ikke mulig å endre attributtene som krever at nettverket leses inn på nytt. Altså disse:

In [ ]:
try:
    G.aar = 2021
    G.kjoretoy = "sykkel"
    G.kommuner = "0301"
    G.noder = None
except AttributeError as e:
    print("AttributeError: ", e)

Men dette funker:

In [ ]:
G = nz.Graf(aar = 2022,
            kjoretoy = "sykkel",
            kommuner = "0301")
G

### kostnad

kostnad er satt til minutter som default, men kan endres til meter. 

In [ ]:
G.kostnad

Man kan også lage egne kostnader og legge det til som kolonner i nettverket. For eksempel hvis man vil finne grønneste rute, mest støyfrie rute eller lignende. 

### fart
Sykkel og til fots er satt til en konstant fart på henholdvis 20 og 5 kilometer i timen:

In [ ]:
G = nz.Graf(kjoretoy = "sykkel")
G.fart

In [ ]:
G = nz.Graf(kjoretoy = "fot")
G.fart

### prosent_straff...

for stigning. Gjelder sykkel og fot. Større straff for oppoverbakker for sykkel, og større gevinst for nedover. 

### forbudte_vegtyper

Dette gjelder kun fot og sykkel. Fortau/gangveg er egentlig lov å sykle på når få fotgjengere, men her antas det at det er rushtid og mange fotgjengere.

In [ ]:
nz.Graf(kjoretoy = "fot").forbudte_vegtyper

In [ ]:
nz.Graf(kjoretoy = "sykkel").forbudte_vegtyper

### sperring
Sperringer gjelder kun når kjoretoy=='bil'. Som default er alle vegbommer med:

In [ ]:
G = nz.Graf()
G.sperring

Men man kan velge at bare noen eller ingen vegkategorier skal ha sperringer (hvis man undersøker kjøretid for utrykning, skogeiere, folk med tilgang til egen privatveg osv.):

In [ ]:
G = nz.Graf(sperring = "ERFK") # nå er det lov å kjøre gjennom private bommer og skogsbilvegbommer
G = nz.Graf(sperring = None) # nå er alle bommer lov å kjøre gjennom
G = nz.Graf() # nå er ingen bommer lov å kjøre gjennom

### fjern_isolerte
Sperringer gjør at en del adresser blir isolert inni vegnettet bak bom, ofte inni borettslag eller boliger med innkjørsel.

Disse små, isolerte nettverkene fjernes som default:

In [ ]:
G.fjern_isolerte

Sånn ser de isolerte ut for et lite område (det største røde området er en kolonihage):

In [ ]:
nett = G.nettverk
nett.loc[nett.isolert != 0, "isolert"] = 1
kartlegg(nett.sjoin(nz.til_gdf(punkter.buffer(1000).iloc[0], crs=25833)), "isolert", cmap="bwr")

### dist_faktor
Ikke alle reiser blir funnet selv om isolerte nettverk fjernes. Noen ganger må man lenger enn nærmeste node for å finne veien. Derfor letes det som default 10 prosent + 10 meter lenger unna enn nærmeste node:

In [ ]:
print("default dist_faktor:", G.dist_faktor)

Som betyr at:

In [ ]:
for meter in [1, 10, 100, 1000]:
    print(f"hvis nærmeste node er {meter} meter unna, letes det innen {int(meter * (1+G.dist_faktor/100) + G.dist_faktor)} meter")

Oftest er forskjellene små med høy dist_faktor, men noen ganger kan rutene hoppe over gjerder, hoppe til andre siden av en motorveien eller lignende.

In [ ]:
G = nz.Graf()

G.dist_faktor = 0
od0 = G.od_cost_matrix(punkter, punkter, id_kolonne="idx")

G.dist_faktor = 50
od50 = G.od_cost_matrix(punkter, punkter, id_kolonne="idx")

od0 = od0.rename(columns={"minutter": "minutter0"})
od50 = od50.rename(columns={"minutter": "minutter50"})

resultater = od0.merge(od50, on = ("fra", "til"))

resultater[["minutter0", "minutter50"]].describe().drop("count")

Hvis man vil ha både nøyaktige og fullstendige resultater, kan det være lurt å først bruke lav dist_faktor, så gjenta for reisene som manglet med høyere dist_faktor. 

Det er ikke mange punkter som mangler med dist_faktor = 0:

In [ ]:
import pandas as pd
import numpy as np

G = nz.Graf()
    
resultater = []
for dist_faktor in [0, 10, 100, 250]:
    
    G.dist_faktor = dist_faktor

    od = G.od_cost_matrix(punkter, punkter, id_kolonne="idx")

    resultat = pd.DataFrame({
        "dist_faktor": dist_faktor,
        "mangler_prosent": len(od[od[G.kostnad].isna()]) / len(od)*100,
        "kostnad_median": np.median(od.loc[~od[G.kostnad].isna(), G.kostnad]),
        "kostnad_mean": np.mean(od.loc[~od[G.kostnad].isna(), G.kostnad]),
                        }, index=[0])
    resultater.append(resultat)

resultater = pd.concat(resultater, axis=0, ignore_index=True)
resultater

De siste 0.2 prosentene er alle til/fra et punkt på Hovedøya. Her kreves det rundt 250 i dist_faktor for å komme seg til fastlandet. dist_faktor=250 bør man unngå å gjøre for noe annet enn ekstreme tilfeller. Eller bare droppe disse tilfellene totalt. 

### kost_til_nodene

Start- og sluttpunktene kobles til vegnettet via nærliggende noder. Fram til nodene beveger man seg utenom vegnettet, i luftlinje. Her er det sannsynlig at det ikke er lov å kjøre bil. Det kan riktignok være mulig+lov å gå og/eller sykle der. Derfor legges det til en kostnad fram til nodene i nettverket. 

Parameteret kost_til_nodene avgjør farten fra start-/sluttpunkt til nodene. Denne farten gjelder for luftlinje fram til noden ganget med 1.5 (fordi det alltid er svinger/ulent terreng i Norge).

kost_til_nodene er som default satt til 5 km/t uavhengig av kjøretøy. Det fordi det antas at man må gå til fots fram til vegnettet. 

Dette vil være for lavt hvis det egentlig finnes en veg som ikke er registrert, eller at andre transportmidler er tilgjengelig (båt, snøskuter...). Da kan man endre parameteret. Kan gi mening å gjøre det bare for de sære tilfellene. 

OBS: hvis man setter en høy fart til nettverket og man har en høy dist_faktor, vil mange reiser unngå vegnettet så mye som mulig, siden det da er raskere å bevege seg i luftlinje*1.5.

In [ ]:
import pandas as pd
import numpy as np

resultater = []
for kost_til_nodene in [0, 30, 10, 5, 3]:
    
    G = nz.Graf()
    G.kost_til_nodene = kost_til_nodene
    
    od = G.od_cost_matrix(punkter, punkter, id_kolonne="idx")

    resultat = pd.DataFrame({
        "kost_til_nodene": kost_til_nodene,
        "kostnad_median": np.median(od.loc[~od[G.kostnad].isna(), G.kostnad]),
        "kostnad_mean": np.mean(od.loc[~od[G.kostnad].isna(), G.kostnad]),
                        }, index=[0])
    resultater.append(resultat)

resultater = pd.concat(resultater, axis=0, ignore_index=True)
resultater

Hvis man beregner reisetid fra og til samme punkt, vil 